In [5]:
from transformers import XLMRobertaForTokenClassification, XLMRobertaTokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Load dataset from CoNLL format
dataset = load_dataset('conll2003')
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

/home/ayalk94/Documents/GitHub/Telegram-based-e-commerce/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ayalk94/Documents/GitHub/Telegram-based-e-commerce/env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'], 
        truncation=True, 
        is_split_into_words=True,
        padding="max_length",  # Ensure the token sequences are padded to the same length
    )

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Get word IDs for tokens
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have word_idx = None
            if word_idx is None:
                label_ids.append(-100)
            # Only label the first subword token of each word
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                # Subword tokens are ignored (-100)
                label_ids.append(-100)
            previous_word_idx = word_idx
        # Add padding to label_ids to match the max_length of tokenized sequences
        label_ids += [-100] * (len(tokenized_inputs["input_ids"][i]) - len(label_ids))
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [8]:




# Load dataset and tokenize
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Define label list
label_list = dataset["train"].features["ner_tags"].feature.names

# Model and training setup
model = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(label_list))

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

# Start training
trainer.train()

# Save fine-tuned model
trainer.save_model("fine_tuned_ner_model")


Map: 100%|██████████| 3250/3250 [00:02<00:00, 1104.18 examples/s]
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/ayalk94/Documents/GitHub/Telegram-based-e-commerce/env/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/2634 [00:00<?, ?it/s]